In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["figure.dpi"] = 100
plt.style.use("bmh")

In [ ]:
os.chdir("..")

In [ ]:
df = pd.read_csv("data/cleaned.csv", index_col=0)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

# Enkodowanie

## Cechy numeryczne

### Licza\_dni\_od\_ost\_kontaktu

In [ ]:
ax = df.liczba_dni_od_ost_kontaktu.plot(kind='hist',bins=1000)
ax.set_xlim(0,50)
ax.set_yscale('log')

In [ ]:
bins = [5,10,15,20,25,30,1000]

In [ ]:
df.liczba_dni_od_ost_kontaktu = np.digitize(df.liczba_dni_od_ost_kontaktu, bins)

### Wiek

In [ ]:
ss = StandardScaler()

In [ ]:
df.wiek.plot(kind='hist')

In [ ]:
df.wiek = ss.fit_transform(np.asarray(df.wiek).reshape(-1,1))

## Dl\_polaczenia

In [ ]:
df.dl_polaczenia.plot(kind='hist')

In [ ]:
df.dl_polaczenia = ss.fit_transform(np.asarray(df.dl_polaczenia).reshape(-1,1))

## Cechy cykliczne

In [ ]:
miesiace = {
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12,
}
df.miesiac = df.miesiac.map(miesiace) / 12 * np.pi
df.miesiac = np.sin(df.miesiac) * np.cos(df.miesiac)

## Cechy kategoryczne (z pominieciem _wyksztalcenie_ i _zawod_, gdyz te beda poddane procesowi _feature enginnering_)

In [ ]:
def pred_categorical_with_knn(df: pd.DataFrame, attrib: str, neighbours=3) -> None:
    model = KNeighborsClassifier(n_neighbors=3)
    X_train = df[~(df[attrib] == "unknown")]
    y_train = X_train[attrib]
    X_train = X_train.drop(attrib, axis=1)

    X_pred = df[df[attrib] == "unknown"]
    y_pred = X_pred[attrib]
    X_pred = X_pred.drop(attrib, axis=1)

    model.fit(X_train, y_train)
    df.loc[X_pred.index, attrib] = model.predict(X_pred)

In [ ]:
df["aktywnie_pracujacy"] = ~df.zawod.isin(["student", "unemployed", "retired"]) + 0

In [ ]:
subset = df.select_dtypes(include="object")
subset = subset[subset.columns.difference(["wyksztalcenie"])]

zawod_array = df.zawod

In [ ]:
encoder = OneHotEncoder()
encoded_df = encoder.fit_transform(subset).toarray()
encoded_df = pd.DataFrame(encoded_df, columns=encoder.get_feature_names_out())
encoded_df = encoded_df.drop("sposob_kontaktu_cellular", axis=1)

df = df.join(encoded_df)
df = df.drop(subset.columns, axis=1)

### Wyksztalcenie (zamiana wartosci _unknown_, przy pomocy algorytmu __KNN__)

In [ ]:
attrib = "wyksztalcenie"

In [ ]:
pred_categorical_with_knn(df, attrib, 3)

In [ ]:
df.wyksztalcenie.value_counts()

In [ ]:
map_wyksztalcenie_do_liczby = {
    "illiterate": 0,
    "basic.4y": 1,
    "basic.6y": 2,
    "basic.9y": 3,
    "high.school": 4,
    "professional.course": 5,
    "university.degree": 6,
}

In [ ]:
df.wyksztalcenie = df.wyksztalcenie.map(map_wyksztalcenie_do_liczby)

### Zawod

In [ ]:
attrib = "zawod"

In [ ]:
subset = df.columns.str.startswith('zawod')
df = df.iloc[:, ~subset]
df[attrib] = zawod_array

In [ ]:
pred_categorical_with_knn(df, attrib)

In [ ]:
df.zawod.value_counts()

In [ ]:
attrib = 'zawod'
df = df.drop(attrib, axis=1).join(pd.get_dummies(df.zawod))